In [2]:
from pulp import LpProblem, LpVariable, LpMaximize, lpSum

from datasets import DF, Load, Save

In [3]:
df = Load(DF.COUNTRIES, raw=True)
df

,Country Code (alpha-2),Country Code (alpha-3),Country,IG,Region
0,AF,AFG,Afghanistan,LIC,Asia
1,DZ,DZA,Algeria,UMIC,Africa
2,AS,ASM,American Samoa,HIC,Oceania
3,AG,ATG,Antigua and Barbuda,HIC,North America
4,AR,ARG,Argentina,UMIC,South America
...,...,...,...,...,...
155,VU,VUT,Vanuatu,LMIC,Oceania
156,VN,VNM,Vietnam,LMIC,Asia
157,YE,YEM,Yemen,LIC,Asia
158,ZM,ZMB,Zambia,LMIC,Africa


### Initial Criteria
**Regions**
- Asia: 24
- Africa: 21
- Europe: 13
- North America: 8
- South America: 7
- Oceania: 7

**Income Groups**
- Low: 24
- Lower-Middle: 24
- Upper-Middle: 16
- High: 16

**Total:** 80

In [4]:
rg_tg = {'Asia': 24, 'Africa': 21, 'Europe': 13, 'North America': 8, 'South America': 7, 'Oceania': 7}
ig_tg = {'LIC': 24, 'LMIC': 24, 'UMIC': 16, 'HIC': 16}
t_tg = 80

In [5]:
problem = LpProblem('ctr-sel', LpMaximize)

ctr_vars = LpVariable.dicts('Select', df['Country'], cat = 'Binary')

problem += lpSum(ctr_vars[c] for c in df['Country'])
problem += lpSum(ctr_vars[c] for c in df['Country']) == t_tg

for rg, tc in rg_tg.items():
    problem += lpSum(ctr_vars[c] for c in df[df['Region'] == rg]['Country']) == tc

for ig, tc in ig_tg.items():
    problem += lpSum(ctr_vars[c] for c in df[df['IG'] == ig]['Country']) == tc

In [6]:
problem.solve()

-1

In [7]:
# Assign 'Select' dummy to the original dataframe
df['Select'] = df['Country'].map({ctr: int(slt.varValue) for ctr, slt in ctr_vars.items()})

ctr_sel = df[df['Select'] == 1]

ctr_sel = ctr_sel.drop(columns = ['Select'])
ctr_sel = ctr_sel.reset_index(drop = True)

ctr_sel

,Country Code (alpha-2),Country Code (alpha-3),Country,IG,Region
0,AF,AFG,Afghanistan,LIC,Asia
1,AR,ARG,Argentina,UMIC,South America
2,BD,BGD,Bangladesh,LMIC,Asia
3,BE,BEL,Belgium,HIC,Europe
4,BZ,BLZ,Belize,UMIC,North America
...,...,...,...,...,...
74,AE,ARE,United Arab Emirates,HIC,Asia
75,GB,GBR,United Kingdom,HIC,Europe
76,VU,VUT,Vanuatu,LMIC,Oceania
77,VN,VNM,Vietnam,LMIC,Asia


### Final Solution
**Regions**
- Asia: 25
- Africa: 20
- Europe: 13
- North America: 6
- South America: 8
- Oceania: 7

**Income Groups**
- Low: 22
- Lower-Middle: 22
- Upper-Middle: 18
- High: 17

**Total:** 79

In [8]:
Save(DF.COUNTRIES, ctr_sel)

### World Map

In [9]:
import plotly.graph_objs as go
import plotly.express as px

In [10]:
fig = px.choropleth(ctr_sel, locations = 'Country Code (alpha-3)', color = 'IG' , hover_name = 'Country', projection = 'natural earth',
                    color_discrete_map = {'LIC': '#934c91', 'LMIC': '#da9bd6','UMIC': '#9dc67e', 'HIC': '#0d8251'})

fig.update_layout(
    title = dict(text = 'Countries Selected', font = dict(size=20), 
                     xanchor = 'center', yanchor = 'middle'),
    font_family = "Helvetica Now Text",
    margin = dict(t=50, b=50, l=10, r=25),
    autosize=True,
    width=1000,
    height=550,
    template='seaborn',
    paper_bgcolor="white",
    legend=dict(
        orientation="h",
        yanchor="auto",
        y=-0.1,
        xanchor="right",
        x=0.7
))

fig.update_layout(margin={"r":20,"t":25,"l":20,"b":25})
fig.update_traces(marker_line_width = 0.6, selector = dict(type='choropleth'), marker_opacity = 0.6)

inc_order = ['LIC', 'LMIC', 'UMIC', 'HIC']

traces_re = []
for inc in inc_order:
    for trace in fig.data:
        if trace.name == inc:
            traces_re.append(trace)

fig.data = traces_re
fig.show()

In [11]:
fig.write_image('../images/world-map.png', scale=3, engine='kaleido')